In [4]:
%pwd

'/Users/user/Sites/DataScience/MLOps/end-to-end-ml'

In [4]:
from pathlib import Path
import os

In [6]:
os.chdir('../')

In [7]:
%pwd

'/Users/user/Sites/DataScience/MLOps/end-to-end-ml'

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [1]:
from box import ConfigBox

In [8]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH ):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)

                create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [11]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size


In [12]:
DataIngestionConfig

__main__.DataIngestionConfig

In [30]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [31]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config) 
    data_ingestion.download_file()
    data_ingestion.extract_zip_file() 
except Exception as e:
    raise e

[2024-04-09 23:10:12,964: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-09 23:10:12,973: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-09 23:10:12,978: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-09 23:10:12,982: INFO: common: created directory at: artifacts]
[2024-04-09 23:10:12,986: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-09 23:10:14,478: INFO: 1156268289: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B508:39B179:3C0CD4E:3EBF6B7:6615B8AD
Accept-Ranges: bytes
Date: Tue, 